In [52]:
### pry-docの読み込み
require "/root/jupyter/vendor/bundle/ruby/2.3.0/gems/pry-doc-0.10.0/lib/pry-doc"
#require "/Users/ftakao2007/jupyter/vendor/bundle/ruby/2.3.0/gems/pry-doc-0.10.0/lib/pry-doc"
### エンコーディングをUTF-8に設定
Encoding.default_external = "UTF-8"

"UTF-8"

# 概要

* Railsでのrspecの使い方について

# インストール

* Gemfile

```
================================================
group :development, :test do
  # Call 'byebug' anywhere in the code to stop execution and get a debugger console
  gem 'byebug', platform: :mri
  gem 'rspec-rails'      # 追加
  gem 'capybara'         # 追加 (findやcheckメソッドが使えるようになる)
end
================================================
```

インストール手順

```
### インストールと初期ファイル作成
$ bundle install --path vendor/bundle
$ bundle exec rails generate rspec:install
================================================
      create  .rspec
      create  spec
      create  spec/spec_helper.rb
      create  spec/rails_helper.rb
================================================

### rails_helper.rb にCapybaraの設定を追記
$ vi spec/rails_helper.rb
================================================
require 'rspec/rails'
require 'capybara/rails'   # 追加
require 'capybara/rspec'   # 追加
...
RSpec.configure do |config|
  config.include Capybara::DSL     # 追加
================================================

### モデルやコントローラを作成したときに自動でRspecファイルも作成されるようにする
### config.generatorsのところを追加
$ vi config/application.rb
================================================
...
module Blog
  class Application < Rails::Application
    # Settings in config/environments/* take precedence over those specified here.
    # Application configuration should go into files in config/initializers
    # -- all .rb files in that directory are automatically loaded.
    config.generators do |g|
      g.test_framework = "rspec"
      #g.controller_specs = false     # controllerのRspecファイルを作成しない場合はコメントアウトを外す
      #g.helper_specs = false     # helperのRspecファイルを作成しない場合はコメントアウトを外す
      #g.view_specs = false     # viewのRspecファイルを作成しない場合はコメントアウトを外す
    end
  end
end
================================================
```

In [53]:
### Gemfileの確認
File.open("Gemfile") do |file|
  puts file.read
end

### Jupyter上からインストールする場合はFullPATHで指定
puts `bundle install --gemfile /root/git_jupyter_notebook/Ruby/Gemfile --path /root/git_jupyter_notebook/Ruby/vendor/bundle`

# frozen_string_literal: true
source "https://rubygems.org"

# gem "rails"

gem 'ruby-ldap', '~> 0.9.11'
gem 'rspec'

Don't run Bundler as root. Bundler can ask for sudo if it is needed, and
installing your bundle as root will break this application for all non-root
users on this machine.
Using diff-lcs 1.3
Using rspec-support 3.6.0
Using ruby-ldap 0.9.19
Using bundler 1.14.6
Using rspec-core 3.6.0
Using rspec-expectations 3.6.0
Using rspec-mocks 3.6.0
Using rspec 3.6.0
Bundle complete! 2 Gemfile dependencies, 8 gems now installed.
Bundled gems are installed into /root/git_jupyter_notebook/Ruby/vendor/bundle.



# テスト対象のファイル

* model
    * app/models/webmail/mail.rb

```
================================================
class Webmail::Mail
...
  def score
    0
  end
...
================================================
```

# テスト実行

In [1]:
### specディレクトリの作成
Dir.mkdir('spec') unless Dir.exist?('spec')

### spec/models/webmail/mail_spec.rbファイルの作成
File.open("spec/models/webmail/mail_spec.rb", "w") do |file|
file.write <<-EOS
require 'rails_helper'

RSpec.describe Webmail::Mail, type: :model do
  describe "#score" do
    it "returns 0 for an all gutter mail" do
      mail = Webmail::Mail.new
      expect(mail.score).to eq(0)
    end
  end
end
EOS
end


### テスト実行
puts `RAILS_ENV=development bundle exec rspec -fd spec/models/webmail/mail_spec.rb`

Errno::ENOENT: No such file or directory @ rb_sysopen - spec/models/webmail/mail_spec.rb

実行ログ例

```
Webmail::Mail
  #score
    returns 0 for an all gutter mail

Finished in 0.01226 seconds (files took 1.92 seconds to load)
1 example, 0 failures
```

# FactoryGirl

* 概要
    * テストデータを自動で作って投入してくれる

## インストール

```
### Gemfileの修正
$ vi Gemfile
================================================
   gem 'rspec-rails'
   gem 'factory_girl_rails'   ### このへんに追加
   gem 'capybara', '~> 2.13'
================================================
$ bundle install --path vendor/bundle

### テスト用DB設定 (developやproductionと同じにしてるとデータが壊される可能性があるので)
$ vi config/database.yml
================================================
test:
...
  database: test_blog
...
================================================

### テストDBに値が保存されるように設定を修正(trueだと保存されない)
### http://qiita.com/a_ishidaaa/items/b17ca8fa1a50ed5c3802
$ vi spec/rails_helper.rb
================================================
  config.use_transactional_fixtures = false
================================================

### DB作成
$ RAILS_ENV=test bundle exec rake db:create
$ RAILS_ENV=test bundle exec rake db:migrate

### FactoryGirl用ディレクトリとファイルの追加
$ mkdir spec/factories
$ vi spec/factories/articles.rb
================================================
FactoryGirl.define do
  factory :article do
    title "MyString"  ### この辺がテストデータ
    text "MyText"
  end
end
================================================

### テストの追加
$ vi spec/models/article_spec.rb
================================================
require 'rails_helper'

RSpec.describe Article, type: :model do
  #pending "add some examples to (or delete) #{__FILE__}"
  describe "#テスト実行" do
    context "ArticleのFactoryを作成した場合" do
      it "テストデータが作成される" do 
        ### factories/articles.rbの値が入る
        article_model = FactoryGirl.create(:article)
        ### 以下にするとtextフィールドの値がhogehogeに上書きされる
        #article_model = FactoryGirl.create(:article, text: "hogehoge")

        ### 実行画面にテストデータを表示する
        p article_model
      end
    end
  end
end
================================================

### テストの実行
$ bundle exec rspec -fd
================================================
...
Article
  #テスト実行
    ArticleのFactoryを作成した場合
#<Article id: 13, title: "MyString", text: "MyText", created_at: "2017-08-15 05:51:58", updated_at: "2017-08-15 05:51:58">
      テストデータが作成される
...
================================================

$ mysql -ujupyter -p -e 'select * from test_blog.articles'
================================================
+----+----------+----------+---------------------+---------------------+
| id | title    | text     | created_at          | updated_at          |
+----+----------+----------+---------------------+---------------------+
| 12 | MyString | hogehoge | 2017-08-15 05:34:54 | 2017-08-15 05:34:54 |  ### 上書きする方で投入したテストデータ
| 13 | MyString | MyText   | 2017-08-15 05:51:58 | 2017-08-15 05:51:58 |
+----+----------+----------+---------------------+---------------------+
================================================

```